In [19]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
import requests

MAX_RANGE = 662
all_urls = []

for i in range(MAX_RANGE):
    url = 'https://www.thehindu.com/search/?q=india%20road_traffic&order=DESC&sort=publishdate&ct=text&s=news&page=' + str(i)
    html_content = requests.get(url).text  # Parse the html content   
    soup = BeautifulSoup(html_content, "html.parser")  # print(soup.prettify()) 
    
    for link in soup.find_all("a", class_="story-card75x1-text"):
        all_urls.append(link.get('href'))
        
all_urls = list(set(lst)) 

In [15]:
df = pd.DataFrame(all_urls, columns = ['links'])
df

,links
0,https://www.thehindu.com/news/national/morning...
1,https://www.thehindu.com/news/cities/Delhi/ped...
2,https://www.thehindu.com/news/cities/bangalore...
3,https://www.thehindu.com/news/cities/Delhi/roa...
4,https://www.thehindu.com/sci-tech/health/india...
...,...
11855,https://www.thehindu.com/news/cities/puducherr...
11856,https://www.thehindu.com/news/cities/bangalore...
11857,https://www.thehindu.com/news/cities/Madurai/c...
11858,https://www.thehindu.com/news/cities/Mangalore...


In [105]:
# Check if the news links has any of these key terms: news, opinion, todays-paper
news, todayspaper, opinion, others = [], [], [], []
for link in df['links']:
    if '/news/' in link:
        news.append(link)
    elif '/todays-paper/' in link:
        todayspaper.append(link)
    elif '/opinion/' in link:
        opinion.append(link)
    else:
        others.append(link)

In [110]:
news_df = pd.DataFrame(news, columns = ['links'])
news_df

,links
0,https://www.thehindu.com/news/national/morning...
1,https://www.thehindu.com/news/cities/Delhi/ped...
2,https://www.thehindu.com/news/cities/bangalore...
3,https://www.thehindu.com/news/cities/Delhi/roa...
4,https://www.thehindu.com/news/cities/bangalore...
...,...
7001,https://www.thehindu.com/news/national/karnata...
7002,https://www.thehindu.com/news/morning-digest-o...
7003,https://www.thehindu.com/news/cities/bangalore...
7004,https://www.thehindu.com/news/cities/Madurai/c...


In [130]:
from bs4 import BeautifulSoup
import requests
headings, locations, timings, texts = [], [], [], []
for index, link in enumerate(news_df['links']):
    html_content = requests.get(link).text  # Parse the html content   
    soup = BeautifulSoup(html_content, "html.parser")  # print(soup.prettify())  
    
    try:
        heading = soup.find('h1', class_=  'title').text.strip().replace('”', '').replace('“', '')
    except:
        heading
    
    location = soup.find('span', {'class': 'blue-color ksl-time-stamp'}).text.strip().replace(':', '').replace(',', '')
    
    try:
        time = soup.find('none').text.strip()
    except:
        time = 'NULL'
    
    headings.append(heading)
    locations.append(location)
    timings.append(time)
    
    try:
        article = soup.find("div", {"class": "paywall"}).findAll('p')
        article_text = ''
        for p in article:
            article_text += ' '.join(p.findAll(text = True))
        texts.append(article_text)
    except:
        texts.append('NULL')
        

In [134]:
final_news_df = pd.DataFrame([list(news_df.links), headings, locations, timings, texts]).T
final_news_df.columns = ['Link', 'Heading', 'Place', 'Date-Time', 'News']
final_news_df

,Link,Heading,Place,Date-Time,News
0,https://www.thehindu.com/news/cities/Delhi/hav...,‘Have never let manpower shortage affect firef...,NEW DELHI,"May 07, 2018 01:30 IST",The first three months of 2018 saw four major ...
1,https://www.thehindu.com/news/cities/Mangalore...,Two killed in road accident in Belthangady,Mangaluru,"December 01, 2020 00:25 IST",The Belthangady police have booked the driver...
2,https://www.thehindu.com/news/national/other-s...,Kairana bypoll: RLD to field ‘united oppositio...,Lucknow,"May 07, 2018 01:36 IST",Buoyed by the Opposition’s grand success in G...
3,https://www.thehindu.com/news/cities/Mangalore...,Creating awareness about taking selfies,Udupi,"February 06, 2017 01:19 IST",Taking selfies and photographs with mobile te...
4,https://www.thehindu.com/news/national/tamil-n...,Problem at post office,May 01 2015 0000 IST,"May 01, 2015 00:00 IST","On April 18, I went to the Erode Collectorate ..."
...,...,...,...,...,...
6001,https://www.thehindu.com/news/national/karnata...,Lok Adalat creates history by settling 2.61 la...,Bengaluru,"December 22, 2020 01:03 IST",The High Court of Karnataka and the Karnataka...
6002,https://www.thehindu.com/news/morning-digest-o...,Morning digest: China changes tack on Kashmir ...,October 09 2019 0759 IST,"October 09, 2019 07:59 IST",China changes tack on KashmirAhead of Chinese ...
6003,https://www.thehindu.com/news/cities/bangalore...,High Court notice to IRDA on false motor vehic...,Bengaluru,"January 28, 2015 00:00 IST",The High Court of Karnataka on Tuesday ordered...
6004,https://www.thehindu.com/news/cities/Madurai/c...,Court orders compensation,MADURAI,"November 24, 2012 12:43 IST",An Additional Sessions Court in Madurai has o...


In [135]:
final_df.to_excel('final_news_1000_7006_df.xlsx')